<a href="https://colab.research.google.com/github/cblh/TorchLeet/blob/main/torch/basic/custom-loss/custom-loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem: Implement Custom Loss Function (Huber Loss)

### Problem Statement
You are tasked with implementing the **Huber Loss** as a custom loss function in PyTorch. The Huber loss is a robust loss function used in regression tasks, less sensitive to outliers than Mean Squared Error (MSE). It transitions between L2 loss (squared error) and L1 loss (absolute error) based on a threshold parameter $ \delta $.

The Huber loss is mathematically defined as:
$$
L_{\delta}(y, \hat{y}) =
\begin{cases}
\frac{1}{2}(y - \hat{y})^2 & \text{for } |y - \hat{y}| \leq \delta, \\
\delta \cdot (|y - \hat{y}| - \frac{1}{2} \delta) & \text{for } |y - \hat{y}| > \delta,
\end{cases}
$$

where:
- $y$ is the true value,
- $\hat{y}$ is the predicted value,
- $\delta$ is a threshold parameter that controls the transition between L1 and L2 loss.

### Requirements
1. **Custom Loss Function**:
   - Implement a class `HuberLoss` inheriting from `torch.nn.Module`.
   - Define the `forward` method to compute the Huber loss as per the formula.

2. **Usage in a Regression Model**:
   - Integrate the custom loss function into a regression training pipeline.
   - Use it to compute and optimize the loss during model training.

### Constraints
- The implementation must handle both scalar and batch inputs for $ y $ (true values) and $ \hat{y} $ (predicted values).


Extra Details: https://en.wikipedia.org/wiki/Huber_loss

<details>
  <summary>💡 Hint</summary>
  Some details: https://www.kaggle.com/code/bigironsphere/loss-function-library-keras-pytorch/notebook
</details>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Generate synthetic data
torch.manual_seed(42)
X = torch.rand(100, 1) * 10  # 100 data points between 0 and 10
y = 2 * X + 3 + torch.randn(100, 1)  # Linear relationship with noise

#TODO: Define the nn.Module for the Huber Loss
class HuberLoss(nn.Module):
    def __init__(self, delta=1.0):
        super(HuberLoss, self).__init__()
        self.delta = delta
    def forward(self, y_pred, y_ture):
        error = torch.abs(y_pred - y_ture)
        loss = torch.where(error <= self.delta, 0.5 * error**2, self.delta * ( error - 0.5 * self.delta))
        return loss.mean()



# Define the Linear Regression Model
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)  # Single input and single output

    def forward(self, x):
        return self.linear(x)

# Initialize the model, loss function, and optimizer
model = LinearRegressionModel()
#TODO: Add the loss
criterion = HuberLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
epochs = 1000
for epoch in range(epochs):
    # Forward pass
    predictions = model(X)
    loss = criterion(predictions, y)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Log progress every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [100/1000], Loss: 0.8869
Epoch [200/1000], Loss: 0.7855
Epoch [300/1000], Loss: 0.6945
Epoch [400/1000], Loss: 0.6134
Epoch [500/1000], Loss: 0.5433
Epoch [600/1000], Loss: 0.4861
Epoch [700/1000], Loss: 0.4404
Epoch [800/1000], Loss: 0.4045
Epoch [900/1000], Loss: 0.3767
Epoch [1000/1000], Loss: 0.3551


In [3]:
# Display the learned parameters
[w, b] = model.linear.parameters()
print(f"Learned weight: {w.item():.4f}, Learned bias: {b.item():.4f}")

# Testing on new data
X_test = torch.tensor([[4.0], [7.0]])
with torch.no_grad():
    predictions = model(X_test)
    print(f"Predictions for {X_test.tolist()}: {predictions.tolist()}")

Learned weight: 2.0713, Learned bias: 2.4650
Predictions for [[4.0], [7.0]]: [[10.750251770019531], [16.964160919189453]]
